<a href="https://colab.research.google.com/github/oxedanda/ipython/blob/main/kiva_analysis_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring Kiva Crowdfunding Loans with Pandas

## Summary
In this notebook, I explore the Kiva Crowdfunding dataset (`kiva_loans.csv`) using pandas.
The objective is to answer six analytical questions related to loan size, currency, sector,
borrower characteristics and time trends.

## Table of Contents
- Load and prepare the data
- Question 1 – Which sectors receive the largest loans?
- Question 2 – How do loan sizes differ across currencies?
- Question 3 – Food-related loans by country and currency
- Question 4 – Borrower gender composition and loan size
- Question 5 – Time trends in a high-volume country
- Question 6 – Who receives the largest 1% of loans?


In [2]:
import pandas as pd

loans = pd.read_csv("kiva_loans.csv")


In [3]:
loans.head()


,id,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,partner_id,posted_time,disbursed_time,funded_time,term_in_months,lender_count,tags,borrower_genders,repayment_interval,date
0,653051,300.0,300.0,Fruits & Vegetables,Food,"To buy seasonal, fresh fruits to sell.",PK,Pakistan,Lahore,PKR,247.0,2014-01-01 06:12:39+00:00,2013-12-17 08:00:00+00:00,2014-01-02 10:06:32+00:00,12.0,12,NaN,female,irregular,2014-01-01
1,653053,575.0,575.0,Rickshaw,Transportation,to repair and maintain the auto rickshaw used ...,PK,Pakistan,Lahore,PKR,247.0,2014-01-01 06:51:08+00:00,2013-12-17 08:00:00+00:00,2014-01-02 09:17:23+00:00,11.0,14,NaN,"female, female",irregular,2014-01-01
2,653068,150.0,150.0,Transportation,Transportation,To repair their old cycle-van and buy another ...,IN,India,Maynaguri,INR,334.0,2014-01-01 09:58:07+00:00,2013-12-17 08:00:00+00:00,2014-01-01 16:01:36+00:00,43.0,6,"user_favorite, user_favorite",female,bullet,2014-01-01
3,653063,200.0,200.0,Embroidery,Arts,to purchase an embroidery machine and a variet...,PK,Pakistan,Lahore,PKR,247.0,2014-01-01 08:03:11+00:00,2013-12-24 08:00:00+00:00,2014-01-01 13:00:00+00:00,11.0,8,NaN,female,irregular,2014-01-01
4,653084,400.0,400.0,Milk Sales,Food,to purchase one buffalo.,PK,Pakistan,Abdul Hakeem,PKR,245.0,2014-01-01 11:53:19+00:00,2013-12-17 08:00:00+00:00,2014-01-01 19:18:51+00:00,14.0,16,NaN,female,monthly,2014-01-01


# Load and prepare the data


In [4]:
# Convert time columns to datetime
loans["posted_time"] = pd.to_datetime(loans["posted_time"], errors="coerce")
loans["disbursed_time"] = pd.to_datetime(loans["disbursed_time"], errors="coerce")
loans["funded_time"] = pd.to_datetime(loans["funded_time"], errors="coerce")

# Extract year from posted_time
loans["posted_year"] = loans["posted_time"].dt.year


In [5]:
loans[["posted_time", "posted_year"]].head()


,posted_time,posted_year
0,2014-01-01 06:12:39+00:00,2014
1,2014-01-01 06:51:08+00:00,2014
2,2014-01-01 09:58:07+00:00,2014
3,2014-01-01 08:03:11+00:00,2014
4,2014-01-01 11:53:19+00:00,2014


In [6]:
# Approximate loan value in USD
# funded_amount is expressed in USD for all loans
loans["loan_amount_usd"] = loans.apply(
    lambda row: row["funded_amount"] if row["funded_amount"] > 0 else row["loan_amount"],
    axis=1
)


In [7]:
loans[["funded_amount", "loan_amount", "currency", "loan_amount_usd"]].head()


,funded_amount,loan_amount,currency,loan_amount_usd
0,300.0,300.0,PKR,300.0
1,575.0,575.0,PKR,575.0
2,150.0,150.0,INR,150.0
3,200.0,200.0,PKR,200.0
4,400.0,400.0,PKR,400.0


In [8]:
def categorize_borrower(genders):
    if not isinstance(genders, str):
        return "N/A"
    parts = set(genders.split(", "))
    if parts == {"female"}:
        return "female"
    elif parts == {"male"}:
        return "male"
    elif "female" in parts and "male" in parts:
        return "mixed_group"
    else:
        return "group"


In [9]:
loans["borrower_type"] = loans["borrower_genders"].apply(lambda x: categorize_borrower(x))


In [10]:
loans[["borrower_genders", "borrower_type"]].head(10)


,borrower_genders,borrower_type
0,female,female
1,"female, female",female
2,female,female
3,female,female
4,female,female
5,female,female
6,female,female
7,female,female
8,female,female
9,female,female


In [11]:
funded = loans[loans["funded_amount"] > 0]


# Question 1 – Which sectors receive the largest loans?

**Objective:** Identify which sectors receive the largest number of loans and the highest total funded amount in USD.


In [12]:
# Group by sector and compute counts and totals
sector_summary = (
    funded.groupby("sector", as_index=False)
    .agg(
        loans_count=("id", "count"),
        total_funded=("funded_amount", "sum"),
        median_funded=("funded_amount", "median")
    )
    .sort_values("total_funded", ascending=False)
)

sector_summary.head(10)


,sector,loans_count,total_funded,median_funded
0,Agriculture,179639,133770635.0,500.0
6,Food,135962,115089210.0,425.0
11,Retail,123875,90864855.0,400.0
12,Services,44549,42966085.0,500.0
2,Clothing,32538,34805270.0,575.0
4,Education,30955,30044920.0,700.0
8,Housing,33597,21377100.0,500.0
10,Personal Use,36281,14189775.0,200.0
1,Arts,11997,11821055.0,475.0
13,Transportation,15396,9842500.0,425.0


**Interpretation:**  
Agriculture, Food, and Retail are the sectors with the highest total funded amount and number of loans.  
This suggests that these sectors dominate Kiva’s portfolio due to high borrower demand, not because individual loans are exceptionally large.  
Median loan amounts across sectors are relatively similar (around 400–500 USD), confirming that frequency drives total funding more than loan size.


# Question 2 – How do loan sizes differ across currencies?

**Objective:** Compare the typical loan size across currencies using the `loan_amount_usd` variable.


In [14]:
# Group by currency
currency_summary = (
    funded.groupby("currency", as_index=False)
    .agg(
        loans_count=("id", "count"),
        mean_loan_usd=("loan_amount_usd", "mean"),
        median_loan_usd=("loan_amount_usd", "median")
    )
)

# Filter to currencies with at least 1000 loans
currency_summary = currency_summary[currency_summary["loans_count"] >= 1000]

# Sort by mean loan size in USD
currency_summary = currency_summary.sort_values("mean_loan_usd", ascending=False)

currency_summary.head(15)


,currency,loans_count,mean_loan_usd,median_loan_usd
44,PYG,11896,2472.482347,2400.0
45,RWF,6691,2265.442385,1425.0
4,BOB,7309,2237.187030,1450.0
32,MMK,1820,1651.565934,950.0
35,MXN,5664,1586.595162,825.0
16,GTQ,7225,1487.245675,925.0
2,AZN,1944,1388.052984,1300.0
41,PEN,21374,1340.921213,900.0
10,CRC,1551,1299.500322,975.0
0,ALL,1921,1270.171786,1175.0


**Interpretation:**  
Loan sizes vary significantly across currencies, even when converted to USD.  
Currencies like PYG and RWF have much higher mean and median loan amounts, indicating that local economic conditions and Kiva partner practices influence typical loan size.  
Lower-value currencies, such as NGN or PHP, correspond to smaller loans, reflecting lower-cost microfinance contexts.


# Question 3 – Food-related loans by country and currency

**Objective:** Identify countries and currencies with the highest median loan sizes for food-related activities.


In [15]:
# Select loans related to Food sector or activity containing "Food"
food_mask = (
    (funded["sector"] == "Food") |
    (funded["activity"].str.contains("Food", case=False, na=False))
)

food_loans = funded[food_mask]

# Group by country and currency
food_country_currency = (
    food_loans.groupby(["country", "currency"], as_index=False)
    .agg(
        loans_count=("id", "count"),
        median_loan_usd=("loan_amount_usd", "median")
    )
)

# Filter to country-currency pairs with at least 100 loans
food_country_currency = food_country_currency[food_country_currency["loans_count"] >= 100]

# Sort by median loan size
food_country_currency = food_country_currency.sort_values("median_loan_usd", ascending=False)

food_country_currency.head(15)


,country,currency,loans_count,median_loan_usd
87,United States,USD,1080,5000.0
61,Paraguay,PYG,2403,3450.0
80,The Democratic Republic of the Congo,USD,1063,3425.0
19,Dominican Republic,DOP,114,2937.5
9,Burundi,BIF,499,2750.0
34,Iraq,USD,105,2500.0
70,Senegal,XOF,1199,2450.0
47,Mexico,MXN,632,2212.5
66,Rwanda,RWF,2258,2050.0
5,Bolivia,BOB,2131,2050.0


**Interpretation:**  
Food-related loans show strong variation by country and currency.  
Countries like the United States and Paraguay have high median loan sizes (>3,000 USD), while others remain smaller.  
This highlights that the same sector (“Food”) can have very different funding scales depending on the local context and partner strategies.


# Question 4 – Borrower gender composition and loan size

**Objective:** Investigate differences in loan size between different borrower types (female, male, mixed, group, N/A).


In [16]:
# Group by borrower_type
borrower_summary = (
    funded.groupby("borrower_type", as_index=False)
    .agg(
        loans_count=("id", "count"),
        mean_funded=("funded_amount", "mean"),
        median_funded=("funded_amount", "median")
    )
    .sort_values("median_funded", ascending=False)
)

borrower_summary


,borrower_type,loans_count,mean_funded,median_funded
3,mixed_group,40279,1821.236873,1050.0
2,male,137371,804.001136,550.0
0,N/A,4092,952.127322,525.0
1,female,486080,699.192869,425.0


**Interpretation:**  
Mixed-gender groups receive the largest loans, followed by male borrowers.  
Female borrowers, while representing the majority, tend to receive smaller loans (~425 USD median).  
This aligns with global microfinance trends: women are frequent borrowers for small-scale projects, whereas mixed or male groups tend to take larger loans.



# Question 5 – Time trends in a high-volume country

**Objective:** Analyze how the number of loans and average loan size evolved over time in a high-volume country (e.g., Philippines).


In [17]:
# Choose a high-volume country
target_country = "Philippines"

country_loans = funded[funded["country"] == target_country]

# Group by posted_year
time_trend = (
    country_loans.groupby("posted_year", as_index=False)
    .agg(
        loans_count=("id", "count"),
        mean_loan_usd=("loan_amount_usd", "mean")
    )
    .sort_values("posted_year")
)

time_trend


,posted_year,loans_count,mean_loan_usd
0,2014,36986,364.836019
1,2015,45867,344.052369
2,2016,48314,331.948814
3,2017,29162,314.248851


**Interpretation:**  
In the Philippines, the number of loans increased from 2014 to 2016, then decreased in 2017, while the average loan size remained stable (~314–345 USD).  
This indicates that total lending volume changes are driven primarily by borrower count, not by changes in individual loan size.


# Question 6 – Who receives the largest 1% of loans?

**Objective:** Identify sectors, currencies, and repayment intervals most common among the largest 1% of loans.


In [19]:
# Compute 99th percentile of loan_amount_usd
p99 = funded["loan_amount_usd"].quantile(0.99)

# Filter top 1% loans
top_loans = funded[funded["loan_amount_usd"] >= p99]

# Sector summary
top_sector = (
    top_loans.groupby("sector", as_index=False)
    .agg(top_loans_count=("id", "count"))
    .sort_values("top_loans_count", ascending=False)
)

# Currency summary
top_currency = (
    top_loans.groupby("currency", as_index=False)
    .agg(top_loans_count=("id", "count"))
    .sort_values("top_loans_count", ascending=False)
)

# Repayment interval summary
top_repayment = (
    top_loans.groupby("repayment_interval", as_index=False)
    .agg(top_loans_count=("id", "count"))
    .sort_values("top_loans_count", ascending=False)
)

top_sector, top_currency, top_repayment


(            sector  top_loans_count
 6             Food             2050
 11          Retail             1390
 0      Agriculture              830
 2         Clothing              801
 12        Services              739
 1             Arts              248
 4        Education              229
 3     Construction              115
 7           Health              101
 13  Transportation               69
 10    Personal Use               61
 9    Manufacturing               55
 8          Housing               26
 5    Entertainment               23
 14       Wholesale               12,
    currency  top_loans_count
 28      USD             2700
 22      PYG              946
 23      RWF              827
 20      PEN              599
 2       BOB              461
 18      MXN              321
 9       GTQ              184
 7       DOP              137
 16      MMK               90
 32      XOF               83
 31      XAF               81
 15      KES               71
 30      WST     

**Interpretation:**  
The largest 1% of loans cluster in sectors like Food, Retail, and Agriculture, and are mostly in USD, PYG, and RWF.  
Repayment intervals are mostly monthly or irregular, reflecting structured business loans or seasonal projects.  
This shows that very large loans are concentrated in capital-intensive sectors and stable financial environments.


# Final Conclusion

This exploratory analysis of the Kiva Crowdfunding dataset shows patterns across sectors, currencies, borrower profiles, and time:

- **Sector trends:** Agriculture, Food, and Retail dominate in both number of loans and total funding.
- **Currency differences:** Loan sizes vary substantially across currencies, even after converting to USD.
- **Food-related loans:** Median loan size varies widely by country and currency.
- **Borrower gender:** Mixed-gender groups receive the largest loans; female borrowers, although the majority, typically receive smaller loans.
- **Temporal trends:** In high-volume countries like the Philippines, the number of loans changes over time, while average loan size remains stable.
- **Top 1% loans:** Large loans cluster in specific sectors, currencies, and repayment intervals.

Overall, these insights illustrate how Kiva adapts lending to local economic conditions, borrower characteristics, and partner strategies.
